# Transfer Learning (전이학습)
- 똑똑한 사람이 만들어 놓은 것을 가져가서 쓰는 것
- imagenet 데이터를 학습한 것을 사용
- mscoco image 데이터도 있음
- 사전학습된 모델: base (CNN부분) + haed(ANN부분)
- base(특징 추출을 잘함)는 그대로 쓰고, head 부분을 재구성
  

## 필요한 라이브러리 로딩

In [1]:
import os, zipfile, pathlib
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# Step 1. Downloading the Dogs vs Cats dataset
## 데이터 준비
- 데이터가 있다면 실행 안해도 됨

In [2]:
# 압축을 풀 위치
extract_root = pathlib.Path("./datas_dnn")

### import project dependencies

In [3]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# Step 2. Dataset Preprocessing

In [4]:
BASE_DIR = extract_root / 'cats_and_dogs_filtered'
train_dir  = BASE_DIR / 'train'
val_dir = BASE_DIR / 'validation'

In [5]:
IMG_SIZE = 128  # 모바일넷 모델 이미지 기준
IMAGE_SHAPE= (IMG_SIZE, IMG_SIZE, 3)
BATCH_SIZE = 128
SEED = 1337

In [6]:
train_ds = keras.utils.image_dataset_from_directory(
    train_dir,
    image_size = (IMG_SIZE, IMG_SIZE),
    batch_size = BATCH_SIZE,
    shuffle=True,
    seed = SEED
 )

Found 2001 files belonging to 2 classes.


In [7]:
val_ds = keras.utils.image_dataset_from_directory(
    val_dir,
    image_size = (IMG_SIZE, IMG_SIZE) ,
    batch_size = BATCH_SIZE,
    shuffle=True,
)

Found 1003 files belonging to 2 classes.


In [8]:
class_names = train_ds.class_names

In [9]:
AUTOTUNE = tf.data.AUTOTUNE

In [10]:
train_ds = train_ds.prefetch(AUTOTUNE)

In [11]:
val_ds = val_ds.prefetch(AUTOTUNE)

In [12]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
        layers.RandomTranslation(0.1, 0.1)
    ],
    name = 'data_augmentation'
)

# Step 3. Building the model: MobileNetV2를 활용
## Loading the pre-trained model (MovileNetV2)

### 1) 사전학습 모델 불러오기

In [13]:
# 사전학습된 모델: base(CNN 부분) + head(ANN 부분)
# include=False: 분류기(FC Layer) 제거, CNN의 feature extractor 부분만 사용
base_model=MobileNetV2(input_shape=IMAGE_SHAPE,include_top=False, weights='imagenet')

### 2) Freezing the base model

In [14]:
# 사전학습된 가중치를 그대로 사용하고, 학습 동안 업데이트하지 않겠다
base_model.trainable=False

### 3) 모델의 head부분 새롭게 정의
Defining the model
- functional API라는 방식으로 트랜스퍼 러닝을 구성

In [15]:
# 데이터 입력층 정의
inputs = keras.Input(shape=IMAGE_SHAPE)

In [16]:
# 데이터 증강
x=data_augmentation(inputs)

In [17]:
# rescaling (모바일넷 학습할 떄 이렇게 했다고 함 -> 메뉴얼에 나와있는내용. 이렇게 했을 때 학습이 잘된다고 함)
x=layers.Rescaling(scale=1.0/127.5, offset=-1.0)(x)

In [18]:
# 베이스 모델에 전처리한 데이터 넣기
# training=False: 해당 모델을 학습 모드가 아니라,예측(inference) 모드로 실행
x=base_model(x, training=False)

In [19]:
# 평균은 flattern과 동일 (각 행렬의 대푯값 하나를 추출)
# FC층에전달하기 전 가장 많이 쓰는 "특징 요약" 방식
# MobileNetV2 전이학습 구조
# CNN -> GlobalAveragePooling2D -> FC
# Flattern을 사용해도 됨
x=layers.GlobalAveragePooling2D()(x)

In [20]:
# fully connection층
x=layers.Dense(128, keras.activations.relu)(x)

In [21]:
# 출력층
# outputs=layers.Dense(1, keras.activations.sigmoid)(x)
outputs=layers.Dense(2, activation='softmax')(x)

In [22]:
# 모델 만들기
model=keras.Model(inputs, outputs)

In [23]:
# base_model.summary()
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 128, 128, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ data_augmentation (Sequential)       │ (None, 128, 128, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ rescaling (Rescaling)                │ (None, 128, 128, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ mobilenetv2_1.00_128 (Functional)    │ (None, 4, 4, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         163,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │             258 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,422,210 (9.24 MB)

 Trainable params: 164,226 (641.51 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

### 4) Compiling the model
- 현업에서 일반적으로 전이학습 시 러닝메이트를 0.0001~0.001을 많이 사용함

In [24]:
# 출력층 sigmoid인 경우
# model.compile(optimizer=keras.optimizers.RMSprop(
#             learning_rate=1e-4),
#             loss=keras.losses.BinaryCrossentropy(),
#             metrics = [ keras.metrics.BinaryAccuracy ])

In [25]:
# softmax일 경우 꼭 해주기
from tensorflow.keras import optimizers, losses, metrics

model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),
    loss=losses.SparseCategoricalCrossentropy(),
    metrics=[metrics.SparseCategoricalAccuracy()]
)

In [26]:
early_stop = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

### 5) Training the model

In [27]:
history=model.fit(train_ds, epochs=25, validation_data=val_ds)

Epoch 1/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - loss: 0.6903 - sparse_categorical_accuracy: 0.6517 - val_loss: 0.3622 - val_sparse_categorical_accuracy: 0.8544
Epoch 2/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 15s 959ms/step - loss: 0.3144 - sparse_categorical_accuracy: 0.8801 - val_loss: 0.1975 - val_sparse_categorical_accuracy: 0.9302
Epoch 3/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 15s 941ms/step - loss: 0.2376 - sparse_categorical_accuracy: 0.9005 - val_loss: 0.1436 - val_sparse_categorical_accuracy: 0.9501
Epoch 4/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 15s 924ms/step - loss: 0.2022 - sparse_categorical_accuracy: 0.9240 - val_loss: 0.1194 - val_sparse_categorical_accuracy: 0.9571
Epoch 5/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 15s 940ms/step - loss: 0.1666 - sparse_categorical_accuracy: 0.9350 - val_loss: 0.1068 - val_sparse_categorical_accuracy: 0.9641
Epoch 6/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 15s 933ms/step - loss: 0.1618 - sparse_categorical_accuracy: 0.9375 - val_loss: 0.0962 - val_sparse_categorical_accuracy: 0.9631
E

### 6) Transfer learning model evaluation

In [28]:
# 8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 501ms/step - loss: 0.0600 - sparse_categorical_accuracy: 0.9771
# [0.059976693242788315, 0.9770687818527222]
model.evaluate(val_ds)

8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 509ms/step - loss: 0.0627 - sparse_categorical_accuracy: 0.9761


[0.06266587972640991, 0.9760717749595642]

## 모델 일반화

In [29]:
import tkinter as tk
from tkinter import filedialog
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

In [ ]:
# 1) 파일 선택 다이얼로그 열기
root = tk.Tk()
root.withdraw()  # GUI 창 숨기기
file_paths = 'datas_dnn/Sunflower_sky_backdrop.jpg'

# 2) 선택한 파일들 예측
for fname in file_paths:
    img = load_img(fname, target_size=(IMG_SIZE, IMG_SIZE), color_mode="rgb")
    x = img_to_array(img)            # (H,W,C)
    x = np.expand_dims(x, axis=0)    # (1,H,W,C)
    x = preprocess_input(x)          # MobileNetV2: [-1,1] 스케일

    probs = model.predict(x, verbose=0)[0]     # shape = (2,)
    pred_index = np.argmax(probs)
    pred_class = class_names[pred_index]
    pred_prob = probs[pred_index]

    fname = fname.split("/")[-1]
    print(f"{fname} -> class={pred_class}, prob={pred_prob:.4f}, full_probs={probs}")